
Step 1: 
Read files from Bronze and enforce schema and save as delta tables in silver layer, 
do some sanity checks, partition the file for faster joins
Read schema from json files

Step 2: 
Read Silver layer tables to do agg's and store data in gold layer

Step 3:
Read final report/starts from gold layer



Create directories to store data in different zone from raw data load to aggregate data 

In [0]:

base_path=dbutils.widgets.get("base_path")
zones=['bronze','silver','gold']

try:
  dbutils.fs.ls(base_path)
  print(f"Base directory  {base_path} already exists.")
except Exception:
  dbutils.fs.mkdirs(base_path)
  print(f"Base directory {base_path} created.")
  
for zone in range(0,len(zones)):
  try:
    dbutils.fs.ls(base_path+'/'+zones[zone])
    print(f"{base_path}"+'/'+zones[zone]+" already exists..." )
  except Exception:
    dbutils.fs.mkdirs(base_path+'/'+zones[zone])
    print(f"{base_path}"+'/'+zones[zone]+" created..." )

#dbutils.fs.mkdirs('dbfs:/FileStore/ipl_data/bronze')
#dbutils.fs.mkdirs('dbfs:/FileStore/ipl_data/silver')
#dbutils.fs.mkdirs('dbfs:/FileStore/ipl_data/gold')

dbfs:/FileStore/ipl_data
Base directory  dbfs:/FileStore/ipl_data already exists.
dbfs:/FileStore/ipl_data/bronze already exists...
dbfs:/FileStore/ipl_data/silver already exists...
dbfs:/FileStore/ipl_data/gold already exists...


Raw data files listing from bronze layer

In [0]:
%fs ls 'dbfs:/FileStore/ipl_data/bronze/'

path,name,size,modificationTime
dbfs:/FileStore/ipl_data/bronze/Ball_By_Ball.csv,Ball_By_Ball.csv,24816834,1730318835000
dbfs:/FileStore/ipl_data/bronze/Match.csv,Match.csv,114194,1730318839000
dbfs:/FileStore/ipl_data/bronze/Player.csv,Player.csv,35160,1730318840000
dbfs:/FileStore/ipl_data/bronze/Player_match.csv,Player_match.csv,2674852,1730318842000
dbfs:/FileStore/ipl_data/bronze/Team.csv,Team.csv,341,1730318843000


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType,DecimalType
import json


# Initialize SparkSession
spark = SparkSession.builder.appName("IPL_Orange_Cap_Stats").getOrCreate()


#schema's
ball_by_ball_stats_schema = StructType([
    StructField("Match_id", IntegerType(), False),
    StructField("Over_id", IntegerType(), True),
    StructField("Ball_id", IntegerType(), True),
    StructField("Innings_No", IntegerType(), True),
    StructField("Team_Batting", IntegerType(), True),
    StructField("Team_Bowling", IntegerType(), True),
    StructField("Striker_Batting_Position", IntegerType(), True),
    StructField("Extra_Type", StringType(), True),
    StructField("Runs_Scored", IntegerType(), True),
    StructField("Extra_runs", IntegerType(), True),
    StructField("Wides", IntegerType(), True),
    StructField("Legbyes", IntegerType(), True),
    StructField("Byes", IntegerType(), True),
    StructField("Noballs", IntegerType(), True),
    StructField("Penalty", IntegerType(), True),
    StructField("Bowler_Extras", IntegerType(), True),
    StructField("Out_type", StringType(), True),
    StructField("Caught", IntegerType(), True),
    StructField("Bowled", IntegerType(), True),
    StructField("Run_out", IntegerType(), True),
    StructField("LBW", IntegerType(), True),
    StructField("Retired_hurt", IntegerType(), True),
    StructField("Stumped", IntegerType(), True),
    StructField("caught_and_bowled", IntegerType(), True),
    StructField("hit_wicket", IntegerType(), True),
    StructField("ObstructingFeild", IntegerType(), True),
    StructField("Bowler_Wicket", IntegerType(), True),
    StructField("Match_Date", DateType(), True),
    StructField("Season", IntegerType(), True),
    StructField("Striker", IntegerType(), True),
    StructField("Non_Striker", IntegerType(), True),
    StructField("Bowler", IntegerType(), True),
    StructField("Player_Out", IntegerType(), True),
    StructField("Fielders", IntegerType(), True),
    StructField("Striker_match_SK", IntegerType(), True),
    StructField("StrikerSK", IntegerType(), True),
    StructField("NonStriker_match_SK", IntegerType(), True),
    StructField("NONStriker_SK", IntegerType(), True),
    StructField("Fielder_match_SK", IntegerType(), True),
    StructField("Fielder_SK", IntegerType(), True),
    StructField("Bowler_match_SK", IntegerType(), True),
    StructField("BOWLER_SK", IntegerType(), True),
    StructField("PlayerOut_match_SK", IntegerType(), True),
    StructField("BattingTeam_SK", IntegerType(), True),
    StructField("BowlingTeam_SK", IntegerType(), True),
    StructField("Keeper_Catch", IntegerType(), True),
    StructField("Player_out_sk", IntegerType(), True),
    StructField("MatchDateSK", IntegerType(), True)  # Changed type from DATA to IntegerType
])

player_match_schema= StructType([
    StructField("Player_match_SK", IntegerType(), False),
    StructField("PlayerMatch_key", DecimalType(20, 0), False),  # DECIMAL(20)
    StructField("Match_Id", IntegerType(), False),
    StructField("Player_Id", IntegerType(), False),
    StructField("Player_Name", StringType(), True),  # VARCHAR(255)
    StructField("DOB", DateType(), True),  # DATE
    StructField("Batting_hand", StringType(), True),  # VARCHAR(50)
    StructField("Bowling_skill", StringType(), True),  # VARCHAR(50)
    StructField("Country_Name", StringType(), True),  # VARCHAR(100)
    StructField("Role_Desc", StringType(), True),  # VARCHAR(100)
    StructField("Player_team", StringType(), True),  # VARCHAR(100)
    StructField("Opposit_Team", StringType(), True),  # VARCHAR(100)
    StructField("Season_year", IntegerType(), True),
    StructField("is_manofThematch", IntegerType(), True),
    StructField("Age_As_on_match", IntegerType(), True),
    StructField("IsPlayers_Team_won", IntegerType(), True),
    StructField("Batting_Status", StringType(), True),  # VARCHAR(50)
    StructField("Bowling_Status", StringType(), True),  # VARCHAR(50)
    StructField("Player_Captain", StringType(), True),  # VARCHAR(50)
    StructField("Opposit_captain", StringType(), True),  # VARCHAR(50)
    StructField("Player_keeper", StringType(), True),  # VARCHAR(50)
    StructField("Opposit_keeper", StringType(), True)  # VARCHAR(50)
])

players_data_schema = StructType([
    StructField("PLAYER_SK", IntegerType(), nullable=True),
    StructField("Player_Id", IntegerType(), nullable=True),
    StructField("Player_Name", StringType(), nullable=True),
    StructField("DOB", DateType(), nullable=True),
    StructField("Batting_hand", StringType(), nullable=True),
    StructField("Bowling_skill", StringType(), nullable=True),
    StructField("Country_Name", StringType(), nullable=True)
])


matches_schema = StructType([
    StructField("Match_SK", IntegerType(), nullable=False),
    StructField("match_id", IntegerType(), nullable=False),
    StructField("Team1", StringType(), nullable=True),
    StructField("Team2", StringType(), nullable=True),
    StructField("match_date", DateType(), nullable=True),
    StructField("Season_Year", IntegerType(), nullable=True),
    StructField("Venue_Name", StringType(), nullable=True),
    StructField("City_Name", StringType(), nullable=True),
    StructField("Country_Name", StringType(), nullable=True),
    StructField("Toss_Winner", StringType(), nullable=True),
    StructField("match_winner", StringType(), nullable=True),
    StructField("Toss_Name", StringType(), nullable=True),
    StructField("Win_Type", StringType(), nullable=True),
    StructField("Outcome_Type", StringType(), nullable=True),
    StructField("ManOfMach", StringType(), nullable=True),
    StructField("Win_Margin", IntegerType(), nullable=True),
    StructField("Country_id", IntegerType(), nullable=True)
])

teams_schema = StructType([
    StructField("team_sk", IntegerType(), nullable=False),
    StructField("team_id", IntegerType(), nullable=False),
    StructField("team_name", StringType(), nullable=True)])



In [0]:
# Path to the CSV file's
dbfs_raw_file_path = dbutils.widgets.get("dbfs_raw_file_path")
dbfs_silver_path=dbutils.widgets.get("dbfs_silver_path")
from pyspark.sql.functions import split, col, expr, concat_ws, when
# Read the CSV file's into a DataFrame


#Enforce schema on when reading from bronze layer

ball_by_ball_stats_df = spark.read.csv(dbfs_raw_file_path +'Ball_By_Ball.csv', header=True, schema=ball_by_ball_stats_schema,mode="FAILFAST")

player_match_df = spark.read.csv(dbfs_raw_file_path +'Player_match.csv', header=True, schema=player_match_schema,mode="FAILFAST")

players_data_df= spark.read.csv(dbfs_raw_file_path +'Player.csv', header=True, schema=players_data_schema,mode="FAILFAST")

matches_df= spark.read.csv(dbfs_raw_file_path +'Match.csv', header=True, schema=matches_schema,mode="PERMISSIVE")

teams_df= spark.read.csv(dbfs_raw_file_path +'Team.csv', header=True, schema=teams_schema,mode="FAILFAST")



## Apply minor transformations like Add new fileds, standardizing column values 


ball_by_ball_stats_df=ball_by_ball_stats_df.withColumn("extra_type",
    when(col("extra_type") == "byes", "Byes")
    .when(col("extra_type") == "legbyes", "Legbyes")
    .when(col("extra_type") == "noballs", "Noballs")
    .otherwise(col("extra_type")))

teams_df=teams_df.withColumn("team_name_abbrivation",
                             expr("concat_ws('', transform(split(team_name, ' '),x->substring(x,1,1)))"))

player_match_df=player_match_df.withColumn("player_team_abbrivation",
                             expr("concat_ws('', transform(split(player_team, ' '),x->substring(x,1,1)))")).withColumn("Opposit_Team_abbrivation",expr("concat_ws('', transform(split(Opposit_Team, ' '),x->substring(x,1,1)))"))



# Write to silver layer as delta tables.
ball_by_ball_stats_df.write.format("delta").mode("overwrite").option("path", dbfs_silver_path+"ball_by_ball_stats").partitionBy('season').saveAsTable("ball_by_ball_stats")

player_match_df.write.format("delta").mode("overwrite").option("path", dbfs_silver_path+"player_match").option("mergeSchema", "true").partitionBy("season_year").saveAsTable("player_match")

players_data_df.write.format("delta").mode("overwrite").option("path",dbfs_silver_path+"players_data").saveAsTable("players_data")

matches_df.write.format("delta").mode("overwrite").option("path",dbfs_silver_path+"matches").saveAsTable("matches")


teams_df.write.format("delta").mode("overwrite").option("path", dbfs_silver_path+"teams").option("mergeSchema", "true").saveAsTable("teams")



In [0]:
%fs ls /FileStore/ipl_data/silver/

path,name,size,modificationTime
dbfs:/FileStore/ipl_data/silver/ball_by_ball_stats/,ball_by_ball_stats/,0,0
dbfs:/FileStore/ipl_data/silver/matches/,matches/,0,0
dbfs:/FileStore/ipl_data/silver/player_match/,player_match/,0,0
dbfs:/FileStore/ipl_data/silver/players_data/,players_data/,0,0
dbfs:/FileStore/ipl_data/silver/teams/,teams/,0,0


In [0]:
%sql
select 'ball_by_ball_stats' as table_name,count(*) as record_count from ball_by_ball_stats
union 
select 'matches' as table_name,count(*) as record_count from matches
union 
select 'player_match' as table_name,count(*) as record_count from player_match
union 
select 'players_data' as table_name,count(*) as record_count from players_data
union
select 'teams' as table_name,count(*) as record_count from teams;

table_name,record_count
ball_by_ball_stats,150451
matches,637
player_match,13992
players_data,498
teams,13
